In [ ]:
# Import needed packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.metrics import roc_curve, auc, roc_auc_score
from scipy.stats import sem
import shap
from xgboost import XGBClassifier

# Define path to the data and read the data into dataframe
data = ".../xgboost_validation_data.csv"
df = pd.read_csv(data)

# Split the data into features (X) and target (y): 
X = df.iloc[:,2:]
y = df.iloc[:,1]

# Load the trained XGBoost model
final_xgb_model = XGBClassifier()
final_xgb_model.load_model('TrainedXGBF.model')

# Predict probabilities and outcomes 
y_predictedProb = final_xgb_model.predict_proba(X)[:,1] # the probability that outcome = 1
y_predictedOutcome = final_xgb_model.predict(X)

# Save the true outcome values
y_true = np.array(y)

# Save case number, their true outcome, predicted probablity and predicted outcome to an external file
with open('ValidationPredictions.csv', 'w') as file:
    file.write("TNRO;outcome;predictedOutcome;predictedProb\n")
    for index in range(df.shape[0]):
        TNRO = df.iloc[index,0]
        outcome = y_true[index]
        predictedOutcome = y_predictedOutcome[index]
        predictedProb = y_predictedProb[index]
        file.write(f"{TNRO};{outcome};{predictedOutcome};{predictedProb}\n")

# Calculate AUC
print("ROC area (AUC): {:0.3f}".format(roc_auc_score(y_true, y_predictedProb)))

# Calculate AUC confidence intervals with bootstrapping 
n_bootstraps = 2000
rng_seed = 42  # control reproducibility
bootstrapped_scores = []
rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(y_predictedProb), len(y_predictedProb))
    if len(np.unique(y_true[indices])) < 2:
        # At least one positive and one negative sample are needed for ROC AUC
        # to be defined: reject the sample
        continue

    score = roc_auc_score(y_true[indices], y_predictedProb[indices])
    bootstrapped_scores.append(score)

# Compute the lower and upper bound of the 95% confidence interval
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()
confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
    confidence_lower, confidence_upper))

# Save predicted probabilities histogram into separate file
plt.hist(y_predictedProb, bins=300)
plt.title('Histogram of predicted probabilities (outcome = 1)')
plt.show()
plt.savefig("PredictedProbHistogram.pdf", format="pdf")

# Plot ROC
xgb_fpr, xgb_tpr, threshold = roc_curve(y_true, y_predictedProb)
auc_xgb = auc(xgb_fpr, xgb_tpr)
plt.figure(figsize=(7, 7))
plt.plot(xgb_fpr, xgb_tpr, linestyle='-', label=f'XGBoost (AUC = {auc_xgb:.3f}, 95% CI {confidence_lower:.3f} - {confidence_upper:.3f})')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend()
plt.savefig("RocAuc.pdf", format="pdf")

# Plot calibration curve
from sklearn.calibration import CalibrationDisplay
disp = CalibrationDisplay.from_estimator(final_xgb_model, X, y)
plt.savefig("CalibrationCurve.pdf", format="pdf")

# Explain the model's predictions using SHAP
shap.initjs()
fig, ax = plt.subplots()
fig.canvas.draw()
shap.summary_plot(shap_values, X, plot_type="bar",show=False)

labels = [item.get_text() for item in ax.get_yticklabels()]
labels[0] = "Proton pump inhibitor purchase, mother during pregnancy"
labels[1] = "Paracetamol purchase, mother during pregnancy"
labels[2] = 'Term child having neonatal respiratory problems requiring treatment'
labels[3] = "Chronic lower respiratory diseases, father"
labels[4] = "Acute upper respiratory infections, mother"
labels[5] = 'Mother tongue other than official languages of Finland'
labels[6] = "Smoking during pregnancy, mother"
labels[7] = 'Sibling hospitalised for viral bronchitis'
labels[8] = 'Other conditions (incl. fear of labour) affecting the pregnancy and the labor'
labels[9] = 'Have older siblings aged >7 years'
labels[10] = 'Birth weight in SD units, compared to reference values'
labels[11] = "Mother's age"
labels[12] = "Father's age"
labels[13] = 'Diseases of middle ear and mastoid, sibling'
labels[14] = 'Any bronchitis in sibling'
labels[15] = 'Have older siblings aged 4-7 years'
labels[16] = 'Male gender'
labels[17] = 'Gestational age at birth'
labels[18] = 'Have older siblings, age less than 4 years'
labels[19] = 'Age during the next epidemic peak in months'

ax.set_yticklabels(labels)
plt.savefig("SHAP.pdf", format="pdf",bbox_inches='tight')

# compute SHAP values
explainer = shap.Explainer(final_xgb_model, X)
shap_values = explainer(X)

shap.plots.beeswarm(shap_values)
shap.plots.beeswarm(shap_values, order=shap_values.abs.max(0))